<a href="https://colab.research.google.com/github/megsdata/sEMG_SupervisedLearning/blob/main/23May10_42samples_classical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
from google.colab import drive
drive.mount('/MSC_THESIS')

Mounted at /MSC_THESIS


In [13]:
%cd /MSC_THESIS/MyDrive/School - Ryerson Graduate/MSC_THESIS
!ls
df = pd.read_excel('30-Apr-2023_COMBINED.xlsx')
df.head()

/MSC_THESIS/MyDrive/School - Ryerson Graduate/MSC_THESIS
23Apr30_42samples_classical.ipynb  deep_learning_experiments.ipynb
23May10_42samples_classical.ipynb  supervised_learning.ipynb
30-Apr-2023_COMBINED.xlsx


,SubjectID,RandomizedGroup_1_ONFirst_2_OFFFirst_,SubjectDataFileNames,DateOfCollection_dd_mm_yy_,Sex_0_Male_1_Female_,Age_years_,Height_cm_,Weight_kg_,DominantL_R_L_,PreviousLowerLimbInjury_toDominantSide___0_no_1_yes_,...,ContractionMean,ContractionVariance,ContractionLength,contractionAOC,meanAbsVal,normalizedMean,normalizedRMS,normalizedStDev,stdError,stdDev
0,20220124M-01,1,20220124M-01_T1ON,24/01/2022,1,23,157,50,R,0,...,113.480884,8290.825064,10201,10.652741,113.480884,0.130835,0.361712,0.104979,0.901524,91.053968
1,20220124M-01,1,20220124M-01_T1ON,24/01/2022,1,23,157,50,R,0,...,12.889825,1137.703458,49676,3.590240,12.889825,0.015035,0.122616,0.039343,0.151336,33.729860
2,20220124M-01,1,20220124M-01_T1ON,24/01/2022,1,23,157,50,R,0,...,115.756119,7034.838019,10300,10.759002,115.756119,0.135018,0.367449,0.097831,0.826434,83.873941
3,20220124M-01,1,20220124M-01_T1ON,24/01/2022,1,23,157,50,R,0,...,12.994307,1161.930534,49879,3.604762,12.994307,0.015193,0.123258,0.039853,0.152627,34.087102
4,20220124M-01,1,20220124M-01_T1ON,24/01/2022,1,23,157,50,R,0,...,111.474475,7354.813448,10300,10.558147,111.474475,0.130332,0.361015,0.100268,0.845020,85.760209


In [31]:
#select the labels and features
labels = df['NPRS_0_10_']
features = df[['RMS', 'ContractionMax', 'ContractionMin', 'ContractionMean', 'ContractionVariance', 'ContractionLength',\
               'contractionAOC', 'meanAbsVal', 'normalizedMean', 'normalizedRMS', 'normalizedStDev', 'stdError', \
               'stdDev']]
print(labels.shape)
print(features.shape)


(672,)
(672, 13)


In [32]:
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2,random_state=17)

In [19]:

from sklearn.linear_model import LogisticRegression
X, y = features, labels
clf = LogisticRegression(random_state=0).fit(X, y)
#clf.predict(y)
#clf.predict_proba(X[:2, :])
clf.score(X, y)

0.6666666666666666

In [21]:
X_train, y_train = features, labels
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


0.35555555555555557

In [33]:
#decision tree
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)
#tree.plot_tree(clf)

0.9703703703703703

In [35]:
#naive bayes
from sklearn.naive_bayes import GaussianNB

# Build a Gaussian Classifier
model = GaussianNB()

# Model training
model.fit(X_train, y_train)

# Predict Output
predicted = model.predict([X_test[6]])

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
)

y_pred = model.predict(X_test)
accuray = accuracy_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test, average="weighted")

print("Accuracy:", accuray)
print("F1 Score:", f1)

Accuracy: 0.6444444444444445
F1 Score: 0.6803333946191089


In [37]:
#random forest
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [41]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.14 degrees.


In [44]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: -inf %.


In [45]:
#k nearest neighbours
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
knn.fit(X_train, y_train)
print("train shape: " + str(X_train.shape))
print("score on test: " + str(knn.score(X_test, y_test)))
print("score on train: "+ str(knn.score(X_train, y_train)))

train shape: (537, 13)
score on test: 0.8962962962962963
score on train: 0.8994413407821229


In [46]:
#SVM AGAIN
from sklearn.svm import LinearSVC
svm=LinearSVC(C=0.0001)
svm.fit(X_train, y_train)
print("score on test: " + str(svm.score(X_test, y_test)))
print("score on train: "+ str(svm.score(X_train, y_train)))

score on test: 0.6370370370370371
score on train: 0.6852886405959032


In [47]:
#bagging decision tree - ensemble 1
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
# max_samples: maximum size 0.5=50% of each sample taken from the full dataset
# max_features: maximum of features 1=100% taken here all 10K 
# n_estimators: number of decision trees 
bg=BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=10)
bg.fit(X_train, y_train)
print("score on test: " + str(bg.score(X_test, y_test)))
print("score on train: "+ str(bg.score(X_train, y_train)))

score on test: 0.8962962962962963
score on train: 0.9776536312849162


In [48]:
#boosting decision tree - ensemble 2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adb = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),n_estimators=10,learning_rate=0.6)
adb.fit(X_train, y_train)
print("score on test: " + str(adb.score(X_test, y_test)))
print("score on train: "+ str(adb.score(X_train, y_train)))

score on test: 0.9555555555555556
score on train: 1.0


In [49]:
#random forests - ensemble 3
from sklearn.ensemble import RandomForestClassifier
# n_estimators = number of decision trees
rf = RandomForestClassifier(n_estimators=30, max_depth=9)
rf.fit(X_train, y_train)
print("score on test: " + str(rf.score(X_test, y_test)))
print("score on train: "+ str(rf.score(X_train, y_train)))

score on test: 0.9259259259259259
score on train: 0.9888268156424581


In [23]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
#graph.render("results") 
dot_data = tree.export_graphviz(clf, out_file=None, 
                     feature_names=features,  
                     class_names=iris.target_names,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

'results.pdf'

In [ ]:
from xgboost import XGBClassifier
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2,random_state=108)
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)
score = xgb_clf.score(X_test, y_test)
#score = xgb_clf.score(X_val, y_val)
print(score)

0.9703703703703703


In [ ]:
from sklearn import tree